# Propensity to Convert ML Model Usage

In this notebook we will be using sample behavioral data collected by Snowplow's Javascript tracker from Snowplow's [website](https://snowplow.io/). 

Using our production propensity model, we will perform predictions on users from this dataset and save the output to a new table to feed into marketing campaigns. 


### Configuration

* Python 3.8
* Install Snowpark for Python and other libraries to your environment
* Update `connection.json` with your Snowflake account details and credentials

In [9]:
import json
import cachetools
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf, call_udf, col

In [4]:
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()

### Perform predictions
Load and use your model to predict propensity to convert.

In [14]:
session.clear_imports()
session.clear_packages()

import imblearn
imblearn_path = imblearn.__path__[0]

session.add_import('@ml_models/model.joblib')
session.add_import(imblearn_path)
session.add_packages('pandas', 'scikit-learn', 'dill')


@cachetools.cached(cache={})
def read_model(filename):
  import dill
  import sys
  import os

  import_dir = sys._xoptions.get("snowflake_import_directory")
  if import_dir:
    with open(os.path.join(import_dir, filename), 'rb') as file:
      m = dill.load(file)
      return m


@udf(
  name='predict_propensity_to_convert',
  session=session,
  replace=True,
  is_permanent=True,
  stage_location='@ml_model_udfs',
  packages=["cachetools", "pandas", "dill", "lightgbm"]
)
def predict_propensity_to_convert(
  first_page_title: str,
  refr_urlhost: str,
  refr_medium: str,
  mkt_medium: str,
  mkt_source: str,
  mkt_term: str,
  mkt_campaign: str,
  engaged_time_in_s: float,
  absolute_time_in_s: float,
  vertical_percentage_scrolled: float,
  geo_country: str,
  geo_region: str,
  br_lang: str,
  device_family: str,
  os_family: str,
) -> float:
  import pandas as pd

  model = read_model('model.joblib')

  df = pd.DataFrame([[
    first_page_title,
    refr_urlhost,
    refr_medium,
    mkt_medium,
    mkt_source,
    mkt_term,
    mkt_campaign,
    engaged_time_in_s,
    absolute_time_in_s,
    vertical_percentage_scrolled,
    geo_country,
    geo_region,
    br_lang,
    device_family,
    os_family]],
    columns=[
      "first_page_title",
      "refr_urlhost",
      "refr_medium",
      "mkt_medium",
      "mkt_source",
      "mkt_term",
      "mkt_campaign",
      "engaged_time_in_s",
      "absolute_time_in_s",
      "vertical_percentage_scrolled",
      "geo_country",
      "geo_region",
      "br_lang",
      "device_family",
      "os_family"]
    )

  prediction = model.predict_proba(df)[0][1]

  return prediction

The version of package pandas in the local environment is 1.5.3, which does not fit the criteria for the requirement pandas. Your UDF might not work when the package version is different between the server and your local environment
The version of package scikit-learn in the local environment is 1.2.1, which does not fit the criteria for the requirement scikit-learn. Your UDF might not work when the package version is different between the server and your local environment
The version of package cachetools in the local environment is 5.3.0, which does not fit the criteria for the requirement cachetools. Your UDF might not work when the package version is different between the server and your local environment
The version of package pandas in the local environment is 1.5.3, which does not fit the criteria for the requirement pandas. Your UDF might not work when the package version is different between the server and your local environment
The version of package lightgbm in the local env

You can now utilize this UDF in SQL:
```sql
SELECT converted_user, predict_propensity_to_convert(first_page_title, refr_urlhost, refr_medium, mkt_medium, mkt_source, mkt_term, mkt_campaign, engaged_time_in_s, absolute_time_in_s, vertical_percentage_scrolled, geo_country, geo_region, br_lang, device_family, os_family) AS prediction
FROM first_touch_user_features
```

Alternatively you can use it in python too.

In [34]:
import pandas as pd

user_features = session.table('FIRST_TOUCH_USER_FEATURES')

predictions = user_features.limit(100).select(
  'user_id', 'first_page_title', 'refr_urlhost', 'refr_medium', 'mkt_medium', 'mkt_source', 'mkt_term', 'mkt_campaign', 'engaged_time_in_s',
  'absolute_time_in_s', 'vertical_percentage_scrolled', 'geo_country', 'geo_region', 'br_lang', 'device_family', 'os_family',
  call_udf(
    'predict_propensity_to_convert', col('first_page_title'), col('refr_urlhost'), col('refr_medium'), col('mkt_medium'), col('mkt_source'),
    col('mkt_term'), col('mkt_campaign'), col('engaged_time_in_s'), col('absolute_time_in_s'), col('vertical_percentage_scrolled'),
    col('geo_country'), col('geo_region'), col('br_lang'), col('device_family'), col('os_family')
  ).as_('propensity_score')
)

predictions = predictions.sort(col("propensity_score"), ascending=False)

predictions_df = pd.DataFrame(predictions.collect())

display(predictions_df.head(20))


,USER_ID,FIRST_PAGE_TITLE,REFR_URLHOST,REFR_MEDIUM,MKT_MEDIUM,MKT_SOURCE,MKT_TERM,MKT_CAMPAIGN,ENGAGED_TIME_IN_S,ABSOLUTE_TIME_IN_S,VERTICAL_PERCENTAGE_SCROLLED,GEO_COUNTRY,GEO_REGION,BR_LANG,DEVICE_FAMILY,OS_FAMILY,PROPENSITY_SCORE
0,00094b5d7449bd89425fc6dede800b55,Get started with your Snowplow BDP demo | Snow...,www.linkedin.com,social,None,None,None,None,75,9,36.0,TR,34,tr-TR,Mac,Mac OS X,0.999991
1,010c456d792b393805049701b1949679,Get in touch with the team at Snowplow | Snowplow,None,None,None,None,None,None,0,0,18.0,US,VA,C,Other,Linux,0.999964
2,02b05906d86306b843299e6c36c38454,Get in touch with the team at Snowplow | Snowplow,www.google.com,search,ppc,google-ad,data monetize,data-utilization,55,79,88.0,US,OR,en-US,Other,Windows,0.999858
3,0138c80b431f2718ed7fc7b25bd0020d,Snowplow: behavioral data creation leader,None,None,None,None,None,None,510,159,13.0,US,NY,en-US,Mac,Mac OS X,0.997642
4,006233e361f71e798c39e187c9d64e7b,Snowplow: behavioral data creation leader,None,None,None,None,None,None,50,0,9.0,US,MA,en-US,Pixel 5a,Android,0.978698
5,017ea266e3b7c5bd139f3efb82023a03,Snowplow: behavioral data creation leader,None,None,None,None,None,None,310,182,31.0,US,VA,ru-RU,Other,Linux,0.957348
6,02164305e914a9159caf852d3ef7cfa6,Snowplow: behavioral data creation leader,None,None,None,None,None,None,20,39,100.0,PK,PB,en-US,Other,Windows,0.949957
7,0247a40e8d5d7ac9d4541bec8d217c90,Book a demo | Snowplow,None,None,None,None,None,None,60,129,99.0,CA,ON,en-US,Samsung SM-G781W,Android,0.444766
8,008942fd19eeae0185c8168ffa957589,Snowplow: behavioral data creation leader,None,None,None,None,None,None,30,18,79.0,EG,None,en-US,Other,Windows,0.045704
9,00ec5d8a1cd304cbfe97fac179b0e659,Snowplow: behavioral data creation leader,www.google.com,search,None,None,None,None,0,0,11.0,GE,TB,en-US,Mac,Mac OS X,0.003009


### Save predictions

Depending on your use case, it can be useful to bucket these propensity scores using deciles or cut them into labels like *High*, *Medium* and *Low* propensity. This makes it easier for data consumers to use these predictions, for example, to filter marketing campaign audiences. Either save your scores to a new table, or add them into your main user table.


In [35]:
predictions_df["PROPENSITY_DECILE"] = pd.qcut(predictions_df["PROPENSITY_SCORE"], 10, labels=False)
predictions_df["PROPENSITY_LABEL"] = pd.cut(predictions_df["PROPENSITY_SCORE"], [0., 0.33, 0.66, 1.0], include_lowest=True,
                                            labels=['Low', 'Medium', 'High'])

In [36]:
import plotly.express as px

fig = px.histogram(predictions_df, x="PROPENSITY_SCORE", color="PROPENSITY_LABEL", nbins=50, log_y=True)
fig.show()

In [37]:
# Save scores to table
df = predictions_df[["USER_ID", "PROPENSITY_SCORE", "PROPENSITY_DECILE", "PROPENSITY_LABEL"]]
snowpark_predictions = session.write_pandas(df, "SNOWPLOW_USER_PROPENSITY_SCORES", auto_create_table=True, overwrite=True)

df.head(20)


,USER_ID,PROPENSITY_SCORE,PROPENSITY_DECILE,PROPENSITY_LABEL
0,00094b5d7449bd89425fc6dede800b55,0.999991,9,High
1,010c456d792b393805049701b1949679,0.999964,9,High
2,02b05906d86306b843299e6c36c38454,0.999858,9,High
3,0138c80b431f2718ed7fc7b25bd0020d,0.997642,9,High
4,006233e361f71e798c39e187c9d64e7b,0.978698,9,High
5,017ea266e3b7c5bd139f3efb82023a03,0.957348,9,High
6,02164305e914a9159caf852d3ef7cfa6,0.949957,9,High
7,0247a40e8d5d7ac9d4541bec8d217c90,0.444766,9,Medium
8,008942fd19eeae0185c8168ffa957589,0.045704,9,Low
9,00ec5d8a1cd304cbfe97fac179b0e659,0.003009,9,Low
